# Project 4 Data Wrangling

## Gather

In this part, we need to gather data from:
- A given url: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv
- Twitter API with Tweepy library

In [12]:
import requests
import os
import pandas as pd
import tweepy
import json
import time

In [33]:
# Gather from downloading manually
df_archive = pd.read_csv('data/twitter-archive-enhanced.csv')
df_archive.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [28]:
# Gather data with a given url
folder_name = 'data'
if not os.path.exists('data/image-predictions.tsv'):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
    response = requests.get(url)
    with open(os.path.join(folder_name, url.split('/')[-1]), mode='wb') as file:
        file.write(response.content)

In [29]:
os.listdir(folder_name)

['twitter-archive-enhanced.csv', 'image-predictions.tsv', 'tweet_json.txt']

In [30]:
df_image = pd.read_csv('data/image-predictions.tsv', '\t')
df_image.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [31]:
# Gather data with tweepy
def gather_from_tweepy(consumer_key, consumer_secret, access_token, access_secret):
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [36]:
if not os.path.exists('data/tweet_json.txt'):
    # Please enter your own keys:
    consumer_key = ''
    consumer_secret = ''
    access_token = ''
    access_secret = ''
    gather_from_tweepy(consumer_key, consumer_secret, access_token, access_secret)
    
    json_list = []
    tweet_deleted = []

    start = time.time()
    for tweet_id in df_archive.tweet_id:
        try:
            json_list.append(api.get_status(tweet_id, tweet_mode = 'extended')._json)
        except Exception as e:
            tweet_deleted.append(tweet_id)
    end = time.time()
    print(end - start) # Print out the processing time
    
    # Store the data 
    with open('data/tweet_json.txt', 'w') as file:
        json.dump(json_list, file)

In [22]:
json_list[0]

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'id': 892420643555336193,
 'id_str': '892420643555336193',
 'full_text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 'truncated': False,
 'display_text_range': [0, 85],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 892420639486877696,
    'id_str': '892420639486877696',
    'indices': [86, 109],
    'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'url': 'https://t.co/MgUWQ76dJU',
    'display_url': 'pic.twitter.com/MgUWQ76dJU',
    'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 540, 'h': 528, 'resize': 'fit'},
     'small': {'w': 540, 'h': 528, 'resize': 'fit'},
     'large': {'w': 

In [18]:
len(tweet_deleted)

25

In [23]:
# Read the data into a list
df_list = []
with open('data/tweet_json.txt') as file:
    json_data = json.load(file)
    for data in json_data:
        tweet_id = data['id']
        retweet_count = data['retweet_count']
        favorite_count = data['favorite_count']
        df_list.append({
            'tweet_id': tweet_id,
            'retweet_count': retweet_count,
            'favorite_count': favorite_count
        })

In [24]:
# Create Dataframe from the above list of dictionaries
df_api = pd.DataFrame(df_list)
df_api.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7352,35007
1,892177421306343426,5480,30322
2,891815181378084864,3624,22800
3,891689557279858688,7535,38277
4,891327558926688256,8120,36550


## Assess
Now we have three DataFrames: 
- from **twitter-archive-enhanced.csv**, we get `df_archive` 
- from **image-predictions.tsv**, we get `df_image`
- from **tweet_json.txt**, we get `df_api`

## Clean

#### Define

#### Code

#### Test